In [ ]:
import cv2
import numpy as np
import os

def randomize_hsv(image, h_shift, s_shift):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv_image)
    
    # Apply the fixed random shift to hue and saturation
    h = (h + h_shift) % 180
    s = np.clip(s + s_shift, 0, 255)

    hsv_image = cv2.merge([h.astype(np.uint8), s.astype(np.uint8), v])

    rgb_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)

    return rgb_image

def randomize_gamma(image, gamma):
    # Apply the fixed gamma value
    lookup_table = np.array([((i / 255) ** (1.0 / gamma)) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, lookup_table)

def randomize_frame(image, h_shift, s_shift, gamma):
    # First apply gamma correction
    image = randomize_gamma(image, gamma)
    # Then apply HSV randomization
    image = randomize_hsv(image, h_shift, s_shift)
    return image

def add_noise_to_video(original_frame, noise_frame, alpha=0.8, beta=0.2):
    """
    Add noise to the original frame using cv2.addWeighted.
    alpha: weight of the original video frame (0.8 for the original video)
    beta: weight of the noise video frame (0.2 for the noise video)
    """
    return cv2.addWeighted(original_frame, alpha, noise_frame, beta, 0)

def process_video(input_video_path, output_video_path, noise_video_path, h_shift, s_shift, gamma):
    # Open the original video
    cap = cv2.VideoCapture(input_video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video {input_video_path}")
        return

    # Open the noise video
    noise_cap = cv2.VideoCapture(noise_video_path)
    if not noise_cap.isOpened():
        print(f"Error: Could not open noise video {noise_video_path}")
        return

    # Get the video's width, height, and frames per second (fps)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (640, 480))  # Resize output video to 640x480

    while True:
        ret, original_frame = cap.read()
        if not ret:
            break  # End of video

        # Resize original frame to 640x480
        original_frame_resized = cv2.resize(original_frame, (640, 480))

        # Read the corresponding noise frame
        ret, noise_frame = noise_cap.read()
        if not ret:
            print(f"Warning: Noise video has fewer frames than the original video.")
            break

        # Resize noise frame to 640x480
        noise_frame_resized = cv2.resize(noise_frame, (640, 480))

        # Convert frames to RGB (OpenCV reads in BGR)
        original_frame_rgb = cv2.cvtColor(original_frame_resized, cv2.COLOR_BGR2RGB)
        noise_frame_rgb = cv2.cvtColor(noise_frame_resized, cv2.COLOR_BGR2RGB)

        # Apply random transformations (HSV and Gamma) to original frame
        processed_frame = randomize_frame(original_frame_rgb, h_shift, s_shift, gamma)

        # Add noise to the processed frame
        final_frame = add_noise_to_video(processed_frame, noise_frame_rgb)

        # Convert back to BGR for saving
        final_frame_bgr = cv2.cvtColor(final_frame, cv2.COLOR_RGB2BGR)

        # Write the frame to the output video
        out.write(final_frame_bgr)

    # Release everything when done
    cap.release()
    noise_cap.release()
    out.release()
    print(f"Processed video saved to {output_video_path}")

def process_folder(input_folder, output_folder, noise_video_path):
    # Iterate over all subdirectories (000-199)
    for i in range(200):
        folder_name = f"{i:03d}"
        folder_path = os.path.join(input_folder, folder_name)

        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist.")
            continue

        # Generate fixed random values for the video group (h_shift, s_shift, gamma)
        h_shift = np.random.uniform(-10, 10)  # Random hue shift value (fixed for the group)
        s_shift = np.random.uniform(-5, 5)    # Random saturation shift value (fixed for the group)
        gamma = np.random.uniform(0.5, 1.5)   # Random gamma correction (fixed for the group)

        print(f"Processing folder {folder_name} with h_shift={h_shift}, s_shift={s_shift}, gamma={gamma}")

        # Process each video (0.mp4 and 1.mp4)
        for video_file in ['0.mp4', '1.mp4']:
            input_video_path = os.path.join(folder_path, video_file)
            output_video_path = os.path.join(output_folder, folder_name, video_file)

            # Make sure the output subfolder exists
            os.makedirs(os.path.dirname(output_video_path), exist_ok=True)

            # Process the video with fixed random values for the group and noise video
            process_video(input_video_path, output_video_path, noise_video_path, h_shift, s_shift, gamma)

# Example usage:
input_folder = 'coffeecup_place'
output_folder = 'coffeecup_place_noisy_10'
noise_video_path = 'global_video.mp4'  # Specify the path to the noise video
# process_folder(input_folder, output_folder, noise_video_path)


In [ ]:
import os
import shutil

def copy_obs_action_json(input_folder, output_folder):
    # Iterate over all subdirectories (000-199)
    for i in range(200):
        folder_name = f"{i:03d}"
        folder_path = os.path.join(input_folder, folder_name)

        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist.")
            continue

        # Define the path to the obs_action.json file
        obs_action_file = os.path.join(folder_path, 'obs_action.json')

        # Check if the obs_action.json file exists
        if os.path.exists(obs_action_file):
            # Define the new path to copy the file
            new_folder_path = os.path.join(output_folder, folder_name)
            os.makedirs(new_folder_path, exist_ok=True)

            # Define the new file path in the output folder
            new_file_path = os.path.join(new_folder_path, 'obs_action.json')

            # Copy the file to the new location
            shutil.copy(obs_action_file, new_file_path)
            print(f"Copied {obs_action_file} to {new_file_path}")
        else:
            print(f"Warning: {obs_action_file} does not exist.")

# Example usage:
# input_folder = '/path/to/your/input/folder'  # Set this to your input folder
# output_folder = '/path/to/your/output/folder'  # Set this to your output folder
copy_obs_action_json(input_folder, output_folder)
